<a href="https://colab.research.google.com/github/KanishkGar/Eva/blob/main/S5/Session_2_Step_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code 2
Target: 
- The previous model had too many parameters for this assignment(17000) so a new model will be made that has <10,000 parameters.
- The previous model was overfitting so I'll need to add batch norm after each layer

Results: 
- Parameters: 9762
- Best Test Accuracy: 99.0%
- Best Training Accuracy: 99.5%

Analysis:
- Batchnorm was added and param count was reduced to below the limit.
- The model is massively overfitting as the difference between train and test is .5%. So, more transforms such as dropout will be needed to fix this overfitting.
- Data transformations could help reduce this overfitting as well


In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
!/opt/bin/nvidia-smi
!nvcc --version

Sat Jan  2 17:45:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # input: 1x28x28
        # output: 8x14x14 
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=1),
            # RF - 3x3
            nn.ReLU(),
            nn.BatchNorm2d(6),
            nn.Conv2d(6, 12, 3, padding=1),
            # RF - 5x5
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Conv2d(12, 6, 1),
            nn.MaxPool2d(2, 2),
            # RF - 10x10
        )
        # input: 8x14x14
        # output: 16x7x7 
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 12, 3, padding=1),
            # RF - 12x12
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Conv2d(12, 24, 3, padding=1),
            # RF - 14x14
            nn.ReLU(),
            nn.BatchNorm2d(24),
            nn.Conv2d(24, 12, 1),
            nn.MaxPool2d(2, 2),
            # RF - 28x28
        )
        # input: 16x7x7
        # output: 32x1x1 
        self.conv3 = nn.Sequential(
            nn.Conv2d(12, 12, 3),
            # RF - 30x30
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Conv2d(12, 12, 3),
            # RF - 32x32
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Conv2d(12, 24, 3)
        )
      
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

In [8]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]              60
              ReLU-2            [-1, 6, 28, 28]               0
       BatchNorm2d-3            [-1, 6, 28, 28]              12
            Conv2d-4           [-1, 12, 28, 28]             660
              ReLU-5           [-1, 12, 28, 28]               0
       BatchNorm2d-6           [-1, 12, 28, 28]              24
            Conv2d-7            [-1, 6, 28, 28]              78
         MaxPool2d-8            [-1, 6, 14, 14]               0
            Conv2d-9           [-1, 12, 14, 14]             660
             ReLU-10           [-1, 12, 14, 14]               0
      BatchNorm2d-11           [-1, 12, 14, 14]              24
           Conv2d-12           [-1, 24, 14, 14]           2,616
             ReLU-13           [-1, 24, 14, 14]               0
      BatchNorm2d-14           [-1, 24,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [9]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        #add more transforms here later
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        #tqdm._instances.clear()



def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.04852546751499176 batch_id=468 Accuracy=94.41: 100%|██████████| 469/469 [00:14<00:00, 31.31it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0520, Accuracy: 9840/10000 (98%)

EPOCH: 2


loss=0.04354843869805336 batch_id=468 Accuracy=98.41: 100%|██████████| 469/469 [00:14<00:00, 31.48it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0344, Accuracy: 9883/10000 (99%)

EPOCH: 10


loss=0.027847228571772575 batch_id=468 Accuracy=99.53: 100%|██████████| 469/469 [00:14<00:00, 31.78it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0338, Accuracy: 9886/10000 (99%)

EPOCH: 11


loss=0.009969029575586319 batch_id=209 Accuracy=99.59:  45%|████▍     | 210/469 [00:06<00:08, 32.33it/s]

KeyboardInterrupt: ignored